In [18]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import functions as F
from pathlib import Path

DATA_DIR = Path("../data_ecf").resolve()
OUTPUT_DIR = Path("../output").resolve()
CONSOMMATIONS_CLEAN_PATH = OUTPUT_DIR / "consommation_clean"
BATIMENTS_PATH = DATA_DIR / "batiments.csv"
CONSOMMATIONS_AGREGEES_PATH = OUTPUT_DIR / "consommations_agregees"
METEO_CLEAN_PATH = OUTPUT_DIR / "meteo_clean.csv"
TARIFS_ENERGIE_PATH = DATA_DIR / "tarifs_energie.csv"
CONSOMMATIONS_ENRICHIES_CSV_PATH = OUTPUT_DIR / "consommations_enrichies.csv"
CONSOMMATIONS_ENRICHIES_PARQUET_PATH = OUTPUT_DIR / "consommations_enrichies"

#### Etape 2.2 : Fusion et enrichissement
- Charger les consommations nettoyees (depuis Parquet)

In [2]:
df_consommations = pd.read_parquet(CONSOMMATIONS_AGREGEES_PATH.as_posix())
df_consommations['timestamp'] = pd.to_datetime(df_consommations['timestamp'])
df_consommations['date'] = pd.to_datetime(df_consommations['date'])
df_consommations['type_energie'] = df_consommations['type_energie'].astype("str")

df_consommations.info()
df_consommations.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7492584 entries, 0 to 7492583
Data columns (total 17 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   batiment_id            object        
 1   timestamp              datetime64[ns]
 2   consommation           float64       
 3   unite                  object        
 4   hour                   int32         
 5   year                   int32         
 6   month                  int32         
 7   date                   datetime64[ns]
 8   nom                    object        
 9   type                   object        
 10  commune                object        
 11  surface_m2             int32         
 12  annee_construction     int32         
 13  classe_energetique     object        
 14  nb_occupants_moyen     int32         
 15  intensite_energetique  float64       
 16  type_energie           object        
dtypes: datetime64[ns](2), float64(2), int32(6), object(7)
memory usag

,batiment_id,timestamp,consommation,unite,hour,year,month,date,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen,intensite_energetique,type_energie
0,BAT0056,2024-01-13 08:00:00,5.23,m3,8,2024,1,2024-01-13,Ecole Nantes 56,ecole,Nantes,1561,1958,G,135,0.003350,eau
1,BAT0001,2024-01-13 04:00:00,0.22,m3,4,2024,1,2024-01-13,Ecole Paris 1,ecole,Paris,1926,1978,E,225,0.000114,eau
2,BAT0056,2024-01-13 17:00:00,3.42,m3,17,2024,1,2024-01-13,Ecole Nantes 56,ecole,Nantes,1561,1958,G,135,0.002191,eau
3,BAT0001,2024-01-13 09:00:00,2.30,m3,9,2024,1,2024-01-13,Ecole Paris 1,ecole,Paris,1926,1978,E,225,0.001194,eau
4,BAT0056,2024-01-13 19:00:00,3.68,m3,19,2024,1,2024-01-13,Ecole Nantes 56,ecole,Nantes,1561,1958,G,135,0.002357,eau
5,BAT0001,2024-01-13 10:00:00,3.81,m3,10,2024,1,2024-01-13,Ecole Paris 1,ecole,Paris,1926,1978,E,225,0.001978,eau
6,BAT0057,2024-01-13 07:00:00,0.82,m3,7,2024,1,2024-01-13,Mairie Nantes 57,mairie,Nantes,1126,1981,F,35,0.000728,eau
7,BAT0001,2024-01-13 13:00:00,2.68,m3,13,2024,1,2024-01-13,Ecole Paris 1,ecole,Paris,1926,1978,E,225,0.001391,eau
8,BAT0057,2024-01-13 11:00:00,1.36,m3,11,2024,1,2024-01-13,Mairie Nantes 57,mairie,Nantes,1126,1981,F,35,0.001208,eau
9,BAT0002,2024-01-13 09:00:00,1.49,m3,9,2024,1,2024-01-13,Ecole Paris 2,ecole,Paris,1156,2004,C,402,0.001289,eau


- Fusionner avec les donnees meteo (sur commune et timestamp arrondi a l'heure)

In [3]:
df_meteo = pd.read_csv(METEO_CLEAN_PATH.as_posix())
df_meteo['timestamp'] = pd.to_datetime(df_meteo['timestamp'])
df_meteo['date'] = pd.to_datetime(df_meteo['date'])

df_meteo.info()
df_meteo.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252612 entries, 0 to 252611
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   commune                  252612 non-null  object        
 1   timestamp                252612 non-null  datetime64[ns]
 2   temperature_c            252612 non-null  float64       
 3   humidite_pct             252612 non-null  float64       
 4   rayonnement_solaire_wm2  252612 non-null  float64       
 5   vitesse_vent_kmh         252612 non-null  float64       
 6   precipitation_mm         252612 non-null  float64       
 7   date                     252612 non-null  datetime64[ns]
 8   day_of_week              252612 non-null  int64         
 9   month                    252612 non-null  int64         
 10  season                   252612 non-null  object        
dtypes: datetime64[ns](2), float64(5), int64(2), object(2)
memory usage: 21.2+ MB


,commune,timestamp,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,date,day_of_week,month,season
0,Bordeaux,2023-01-01 00:00:00,8.5,43.9,0.8,0.2,0.0,2023-01-01,6,1,Hiver
1,Bordeaux,2023-01-01 01:00:00,2.7,39.7,6.1,21.5,5.3,2023-01-01,6,1,Hiver
2,Bordeaux,2023-01-01 02:00:00,6.2,78.9,49.5,13.1,0.0,2023-01-01,6,1,Hiver
3,Bordeaux,2023-01-01 03:00:00,10.3,64.2,24.3,0.6,14.7,2023-01-01,6,1,Hiver
4,Bordeaux,2023-01-01 04:00:00,0.9,36.4,20.7,35.6,0.0,2023-01-01,6,1,Hiver
5,Bordeaux,2023-01-01 05:00:00,7.9,66.0,5.9,0.3,0.0,2023-01-01,6,1,Hiver
6,Bordeaux,2023-01-01 06:00:00,1.6,40.2,392.3,24.3,0.0,2023-01-01,6,1,Hiver
7,Bordeaux,2023-01-01 07:00:00,4.9,87.9,781.5,27.9,0.0,2023-01-01,6,1,Hiver
8,Bordeaux,2023-01-01 08:00:00,8.4,43.4,565.0,38.8,0.1,2023-01-01,6,1,Hiver
9,Bordeaux,2023-01-01 09:00:00,10.1,83.8,52.3,25.4,0.0,2023-01-01,6,1,Hiver


In [4]:
df_consommations_meteo = df_consommations.merge(
        df_meteo,
        on=["commune", "timestamp", "date", "month"],
        how="left",
)

print(f"  Nombre de colonnes après fusion : {len(df_consommations_meteo.columns)}")
print(df_consommations_meteo.columns)

  Nombre de colonnes après fusion : 24
Index(['batiment_id', 'timestamp', 'consommation', 'unite', 'hour', 'year',
       'month', 'date', 'nom', 'type', 'commune', 'surface_m2',
       'annee_construction', 'classe_energetique', 'nb_occupants_moyen',
       'intensite_energetique', 'type_energie', 'temperature_c',
       'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh',
       'precipitation_mm', 'day_of_week', 'season'],
      dtype='object')


- Fusionner avec le referentiel batiments

La fusion avec le référentiel batiements a déjà été réalisée dans le fichier `03_agregations_spark.py`.

- Fusionner avec les tarifs pour calculer le cout financier

In [5]:
df_tarifs_energie = pd.read_csv(TARIFS_ENERGIE_PATH.as_posix())
df_tarifs_energie['date_debut'] = pd.to_datetime(df_tarifs_energie['date_debut'])
df_tarifs_energie['date_fin'] = pd.to_datetime(df_tarifs_energie['date_fin'])
df_tarifs_energie['type_energie'] = df_tarifs_energie['type_energie'].astype("str")

df_tarifs_energie.info()
df_tarifs_energie.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date_debut      10 non-null     datetime64[ns]
 1   date_fin        10 non-null     datetime64[ns]
 2   type_energie    10 non-null     object        
 3   tarif_unitaire  10 non-null     float64       
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 452.0+ bytes


,date_debut,date_fin,type_energie,tarif_unitaire
0,2023-01-01,2023-06-30,electricite,0.18
1,2023-07-01,2023-12-31,electricite,0.20
2,2023-01-01,2023-06-30,gaz,0.09
3,2023-07-01,2023-12-31,gaz,0.10
4,2023-01-01,2023-12-31,eau,3.50
5,2024-01-01,2024-06-30,electricite,0.21
6,2024-07-01,2024-12-31,electricite,0.22
7,2024-01-01,2024-06-30,gaz,0.11
8,2024-07-01,2024-12-31,gaz,0.12
9,2024-01-01,2024-12-31,eau,3.75


In [6]:
df_merge = (
    df_consommations_meteo
    .merge(df_tarifs_energie, on="type_energie", how="left")
    .query("date >= date_debut and date <= date_fin")
)

print(f"  Nombre de colonnes après fusion : {len(df_merge.columns)}")
print(df_merge.columns)
print(f"  Nombre de lignes avant jointure : {len(df_consommations_meteo)}")
print(f"  Nombre de lignes après jointure : {len(df_merge)}")

  Nombre de colonnes après fusion : 27
Index(['batiment_id', 'timestamp', 'consommation', 'unite', 'hour', 'year',
       'month', 'date', 'nom', 'type', 'commune', 'surface_m2',
       'annee_construction', 'classe_energetique', 'nb_occupants_moyen',
       'intensite_energetique', 'type_energie', 'temperature_c',
       'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh',
       'precipitation_mm', 'day_of_week', 'season', 'date_debut', 'date_fin',
       'tarif_unitaire'],
      dtype='object')
  Nombre de lignes avant jointure : 7492584
  Nombre de lignes après jointure : 7492584


- Creer des features derivees :
  - Consommation par occupant
  - Consommation par m2
  - Cout journalier, mensuel, annuel
  - Indice de performance energetique (IPE)
  - Ecart a la moyenne de la categorie

In [ ]:
df_features = df_merge.copy()

# Consommation par occupant
df_features["consommation_par_occupant"] = df_features["consommation"] / df_features["nb_occupants_moyen"]

# Consommation par m2
df_features["consommation_par_m2"] = df_features["consommation"] / df_features["surface_m2"]

# Cout journalier, mensuel et annuel
df_features["tarif"] = df_features["consommation"] * df_features["tarif_unitaire"]
cout_journalier = df_features.groupby(["batiment_id", "date"]).agg(cout_journialier=("tarif", "sum"))
cout_mensuel = df_features.groupby(["batiment_id", "year", "month"]).agg(cout_mensuel=("tarif", "sum"))
cout_annuel = df_features.groupby(["batiment_id", "year"]).agg(cout_annuel=("tarif", "sum"))
df_features = (
    df_features
    .join(cout_journalier, ["batiment_id", "date"], "left")
    .join(cout_mensuel, ["batiment_id", "year", "month"], "left")
    .join(cout_annuel, ["batiment_id", "year"], "left")
)

# Indice de performance énergétique
consommation_annuelle = df_features.groupby(["batiment_id", "year"]).agg(consommation_annuelle=("consommation", "sum"))
df_features = df_features.join(consommation_annuelle, ["batiment_id", "year"], "left")
df_features["IPE"] = df_features["consommation_annuelle"] / df_features["surface_m2"]

# Ecart à la moyenne des catégorie
consommation_moyenne_par_type = df_features.groupby("type").agg(consommation_moyenne_par_type=("consommation", "mean"))
df_features = df_features.join(consommation_moyenne_par_type, "type", "left")
df_features["ecart_conso_moyenne_type"] = df_features["consommation"] / df_features["consommation_moyenne_par_type"]

print(f"  Nombre de colonnes après ajout des features : {len(df_merge.columns)}")
print(df_features.columns)

  Nombre de colonnes après ajout des features : 27
Index(['batiment_id', 'timestamp', 'consommation', 'unite', 'hour', 'year',
       'month', 'date', 'nom', 'type', 'commune', 'surface_m2',
       'annee_construction', 'classe_energetique', 'nb_occupants_moyen',
       'intensite_energetique', 'type_energie', 'temperature_c',
       'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh',
       'precipitation_mm', 'day_of_week', 'season', 'date_debut', 'date_fin',
       'tarif_unitaire', 'consommation_par_occupant', 'consommation_par_m2',
       'tarif', 'cout_journialier', 'cout_mensuel', 'cout_annuel',
       'consommation_annuelle', 'IPE', 'consommation_moyenne_par_type',
       'ecart_conso_moyenne_type'],
      dtype='object')


In [19]:
# Sauvegarde CSV
df_features.to_csv(CONSOMMATIONS_ENRICHIES_CSV_PATH.as_posix(), index=False)

In [21]:
# Sauvegarde PARQUET
df_features.to_parquet(CONSOMMATIONS_ENRICHIES_PARQUET_PATH.as_posix(), index=False)

- Dictionnaire de donnees (description de toutes les colonnes)

Colonne | Description
:- | :-
'batiment_id' | Identifiant des bâtiments (BAT00056)
'timestamp' | Timestamp de la mesure de consommation (2024-01-13 08:00:00)
'consommation' | Consommation énergétique
'unite' | Unité de la mesure ('m3' 'kWh')
'hour' | Heure de la mesure
'year' | Année de la mesure
'month' | Mois de la mesure
'date' | Date de la mesure, format 2024-01-13
'nom' | Nom du bâtiment
'type' | Type de bâtiment ('ecole' 'mairie' 'mediatheque' 'gymnase' 'piscine')
'commune' | Nom de la commune
'surface_m2' | suface en m2 du bâtiment
'annee_construction' | année de construction du bâtiment
'classe_energetique' | Classe énergétique du bâtiment
'nb_occupants_moyen' | Nombre d'occupants moyen du bâtiment
'intensite_energetique' | Quotient consommation / surface_m2
'type_energie' | Type d'énergie ('eau' 'electricite' 'gaz')
'temperature_c' | Température météo correspondant au timestamp
'humidite_pct' | Humidité correspondante au timestamp
'rayonnement_solaire_wm2' | Rayonnement correspondant au timestamp
'vitesse_vent_kmh' | Vitesse du vent correspondant au timestamp
'precipitation_mm' | Précipitation correspondant au timestamp
'day_of_week' | Jour le de semaine de la mesure
'season' | Saison de la mesure
'date_debut' | Date de début du tarif unitaire
'date_fin' | Date de fin du tarif unitaire
'tarif_unitaire' | Tarif unitaire
'consommation_par_occupant' | Quotient consommation / nb_occupants_moyen
'consommation_par_m2' | Quotient consommation / surface_m2
'tarif' | Produit consommation * tarif_unitaire
'cout_journialier' | Coût journalier d'un bâtiment
'cout_mensuel' | Coût mentuel d'un bâtiment
'cout_annuel' | Coût annuel d'un bâtiment
'consommation_annuelle' | Consommation annuelle d'un bâtiment
'IPE' | Indice de performance énergétique, quotient consommation_annuelle / surface_m2
'consommation_moyenne_par_type' | Consommation annuelle par type de bâtiment
'ecart_conso_moyenne_type' | Ecart à la consommation annuelle par type de bâtiment